In [1]:
import numpy as np
from gensim.models import Word2Vec
from gensim import models

### Word2Vec

In [2]:
word2vec = models.KeyedVectors.load_word2vec_format("/Users/bubuhuy/Downloads/sentiment-classfication-exp-japanese/embeddings/cc.ja.300.vec", binary=False)

In [6]:
word2vec.wv.vocab

/Users/bubuhuy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{'の': <gensim.models.keyedvectors.Vocab at 0x1ae1a70650>,
 '、': <gensim.models.keyedvectors.Vocab at 0x1ae1a67b50>,
 '。': <gensim.models.keyedvectors.Vocab at 0x1ae1a70690>,
 'に': <gensim.models.keyedvectors.Vocab at 0x1ae1a67890>,
 'は': <gensim.models.keyedvectors.Vocab at 0x1ae1a706d0>,
 'が': <gensim.models.keyedvectors.Vocab at 0x1ae1a670d0>,
 'を': <gensim.models.keyedvectors.Vocab at 0x1ae1a70750>,
 'て': <gensim.models.keyedvectors.Vocab at 0x1ae1a67050>,
 'た': <gensim.models.keyedvectors.Vocab at 0x1ae1a70790>,
 'で': <gensim.models.keyedvectors.Vocab at 0x1ae1a674d0>,
 '</s>': <gensim.models.keyedvectors.Vocab at 0x1ae1a70710>,
 'と': <gensim.models.keyedvectors.Vocab at 0x1ae1a67b90>,
 'し': <gensim.models.keyedvectors.Vocab at 0x1ae1a707d0>,
 '・': <gensim.models.keyedvectors.Vocab at 0x1ae1a67bd0>,
 'も': <gensim.models.keyedvectors.Vocab at 0x1ae1a70810>,
 ':': <gensim.models.keyedvectors.Vocab at 0x1ae1a67c10>,
 '）': <gensim.models.keyedvectors.Vocab at 0x1ae1a70850>,
 '/': <gens

In [ ]:
from gensim.similarities.index import AnnoyIndexer

annoy_index = AnnoyIndexer(word2vec, 100)

In [ ]:
approximate_neighbors = word2vec.wv.most_similar([vector], topn=2, indexer=annoy_index)

In [ ]:
print("Approximate Neighbors")
for neighbor in approximate_neighbors:
    print(neighbor)

In [ ]:
normal_neighbors = word2vec.wv.most_similar([vector], topn=11)
print("\nNormal (not Annoy-indexed) Neighbors")
for neighbor in normal_neighbors:
    print(neighbor)

In [ ]:
model = word2vec

In [ ]:
# Set up the model and vector that we are using in the comparison
# model.init_sims()
# annoy_index = AnnoyIndexer(model, 100)

# Dry run to make sure both indices are fully in RAM
vector = model.wv.vectors_norm[0]
model.wv.most_similar([vector], topn=5, indexer=annoy_index)
model.wv.most_similar([vector], topn=5)

import time
import numpy as np

def avg_query_time(annoy_index=None, queries=1000):
    """
    Average query time of a most_similar method over 1000 random queries,
    uses annoy if given an indexer
    """
    total_time = 0
    for _ in range(queries):
        rand_vec = model.wv.vectors_norm[np.random.randint(0, len(model.wv.vocab))]
        start_time = time.process_time()
        model.wv.most_similar([rand_vec], topn=5, indexer=annoy_index)
        total_time += time.process_time() - start_time
    return total_time / queries

queries = 10000

gensim_time = avg_query_time(queries=queries)
annoy_time = avg_query_time(annoy_index, queries=queries)
print("Gensim (s/query):\t{0:.5f}".format(gensim_time))
print("Annoy (s/query):\t{0:.5f}".format(annoy_time))
speed_improvement = gensim_time / annoy_time
print ("\nAnnoy is {0:.2f} times faster on average on this particular run".format(speed_improvement))

### Save Persisting indices to disk

In [ ]:
fname = '/Users/bubuhuy/Downloads/sentiment-classfication-exp-japanese/embeddings/annoy.index'

# Persist index to disk
annoy_index.save(fname)

In [ ]:
# Load index back
fname = '/Users/bubuhuy/Downloads/sentiment-classfication-exp-japanese/embeddings/annoy.index'
import os.path
if os.path.exists(fname):
    annoy_index2 = AnnoyIndexer()
    annoy_index2.load(fname)
    annoy_index2.model = model

In [15]:
# Results should be identical to above
vector = model.wv["ngon"]
approximate_neighbors2 = model.wv.most_similar([vector], topn=2, indexer=annoy_index2)
for neighbor in approximate_neighbors2:
    print(neighbor)

assert approximate_neighbors == approximate_neighbors2

/Users/bubuhuy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/Users/bubuhuy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


('ngon', 0.999999578531515)
('thơm_ngon', 0.5760951042175293)


NameError: name 'approximate_neighbors' is not defined

In [27]:
normal_neighbors3 = model.wv.most_similar([vector], topn=11)
print("\nNormal (not Annoy-indexed) Neighbors")
for neighbor in normal_neighbors3:
    print(neighbor)

/Users/bubuhuy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.



Normal (not Annoy-indexed) Neighbors
('ngon', 1.0)
('thơm_ngon', 0.6406089663505554)
('tuyệt_ngon', 0.6147646307945251)
('ngon_miệng', 0.612115740776062)
('lạ_miệng', 0.592548131942749)
('bổ_dưỡng', 0.5548734664916992)
('thanh_mát', 0.5477113723754883)
('cơm_rang', 0.5444960594177246)
('nếp_cẩm', 0.5392052531242371)
('ngon_lành', 0.538558840751648)
('ngon_mắt', 0.5344654321670532)


### Save memory via memory-mapping indices saved to disk